### To do:

- [x] Sustituir No en var 2
- [x] Ordenar por fecha
- [x] Ordenar campañas
- [ ] Optuna

In [1]:
# dataset interface should be {'train', 'dev', 'test'?, 'X_column', 'label_column'}
import pandas as pd
from hamison_datasets.UPV.load_upv import load_upv_dataset

upv = load_upv_dataset()
# upv_notmapped = load_upv_dataset(map_values=False)

ds = upv[upv['text'].notnull()]
campañas = upv['¿Qué campaña?'].unique().tolist()

upv_features = ['1. Espacio',
                '2. Texto', '3. Multimedia', '4. Elemento desinformador',
                '5.  Tipo multimedia', '6. Alteración multimedia',
                '7. Cuenta emisora', '8. Fuente',
                '9. Protagonista', '10. Atributo',
                '11. Macro tema', '12. Populismo',
                '13. Ataque', '14. Tipo de ataque']
upv_features = [c for c in upv.columns if '.' in c]
# for feat in upv_features:
#     print(upv[feat].value_counts(), '\nTotal:',
#           upv[(upv['text'].notnull() & upv[feat].notnull())].shape[0], '\n')
upv.head(3)

Loaded UPV dataset from UPV_allURLs.csv
Loading html files...
Removing rows without html: [6668]


Extracting articles from HTML:   0%|          | 0/408 [00:00<?, ?it/s]

,Medio,Título desmentido,URL,Titular enlazado,¿Periodo de campaña?,¿Qué campaña?,¿Campaña?,Único,Duplicado misma campaña,Corpus,...,12. Populismo,13. Ataque,14. Tipo de ataque,5. Tipo multimedia reduced,7. Cuenta emisora reduced,9. Protagonista reduced,14. Tipo de ataque reduced,html,article,text
ID_Medio_Crono,,,,,,,,,,,,,,,,,,,,,
429,Maldito dato,"Es falso que Vox esté en el Gobierno ""en Casti...",https://maldita.es/malditodato/20191111/es-fal...,"Es falso que Vox esté en el Gobierno ""en Casti...",Sí,Gen19,Sí,Sí,No,Sí,...,Crítica a las élites,Sí,Electoral,NaN,Clase política de izquierdas,"Gobiernos, partidos y dirigentes de derecha",Electoral,"\n<!DOCTYPE html>\n\n<html lang=""es"">\n<head>\...","[\n, [\n, [\nMALDITO\n, <strong class=""section...",MALDITO\nDATO\n\nEs falso que Vox esté en el G...
258,Newtral Fact-checks,"""(Pablo) Iglesias ha votado 4 veces en contra ...",https://www.newtral.es/sanchez-iglesias-ha-vot...,"""(Pablo) Iglesias ha votado 4 veces en contra ...",Sí,Gen19,Sí,Sí,No,Sí,...,Defensa o apelación al pueblo,Sí,Antecedentes,NaN,Clase política de izquierdas,"Gobiernos, partidos y dirigentes de izquierda",Otros,"<!DOCTYPE html>\n<html lang=""es""><head><meta c...","[[\n, [ , <a class=""author vcard"" href=""https:...",Por María Blanco05 noviembre 2019 | 2 min l...
265,Newtral Fact-checks,"""Estáis gobernando conjuntamente con Vox en la...",https://www.newtral.es/maria-jesus-montero-sob...,"""Estáis gobernando conjuntamente con Vox en la...",Sí,Gen19,Sí,Sí,No,Sí,...,Ostracismo,Sí,Asociación,NaN,Clase política de izquierdas,"Gobiernos, partidos y dirigentes de derecha",Otros,"<!DOCTYPE html>\n<html lang=""es""><head><meta c...","[[\n, [ , <a class=""author vcard"" href=""https:...",Por Lorena Baeza08 noviembre 2019 | < 1 min...


In [2]:
# classfier pipeline and metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline, make_pipeline
from stopwordsiso import stopwords
import numpy as np
import json

stop_words = list(stopwords('es'))

tfidf_vectorizer = TfidfVectorizer(
    stop_words=stop_words, use_idf=True, sublinear_tf=True, ngram_range=(1, 2), min_df=3, max_df=0.8)
clf = RandomForestClassifier()
pipe = make_pipeline(tfidf_vectorizer, clf)

In [1]:
import optuna


# classfier pipeline and metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline, make_pipeline
from stopwordsiso import stopwords
import numpy as np
import json

stop_words = list(stopwords('es'))


def objective_fn(trial):

    x, y = data, labels

    stop_words = trial.suggest_categorical("stop_words", [True, False])
    use_idf = trial.suggest_categorical("use_idf", [True, False])
    sublinear_tf = trial.suggest_categorical("sublinear_tf", [True, False])
    min_df = trial.suggest_int("min_df", 1, 10, step=1)
    max_df = trial.suggest_int("max_df", 0, 1, step=0.1)
    ngram_range = trial.suggest_categorical("ngram_range", [
        (1, 1), (1, 2), (1, 3), (1, 4), (1, 5),
        (2, 2), (2, 3), (2, 4), (2, 5),
        (3, 3)])

    tfidf_vectorizer = TfidfVectorizer(
        stop_words=stop_words, use_idf=use_idf, sublinear_tf=sublinear_tf, ngram_range=ngram_range, min_df=min_df, max_df=max_df)

    classifier_name = trial.suggest_categorical(
        "classifier", ["SVC", "RandomForest"])
    if classifier_name == "SVC":
        svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
        clf = SVC(C=svc_c, gamma="auto")
    else:
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        clf = RandomForestClassifier(
            max_depth=rf_max_depth, n_estimators=10
        )
    pipe = make_pipeline(tfidf_vectorizer, clf)

    skf = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
    skf.get_n_splits(data, data[feat])

    cv_f1_scores = cross_val_score(
        estimator=pipe, X=x, y=y,
        n_jobs=-1, cv=skf, scoring='f1_macro')
    cv_f1_avg = cv_f1_scores.mean()
    return cv_f1_avg


for feat in upv_features:
    # print(feat)
    for camp, data in upv.groupby('¿Qué campaña?'):
        study = optuna.create_study(direction="maximize")
        study.optimize(objective_fn, n_trials=100)
        print(study.best_trial)

In [ ]:
import optuna


# classfier pipeline and metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline, make_pipeline
from stopwordsiso import stopwords
import numpy as np
import json

stop_words = list(stopwords('es'))


class Objective:
    def __init__(self, label_col):
        # Hold this implementation specific arguments as the fields of the class.
        self.x = data['text']
        self.y = data[label_col]

    def __call__(self, trial):
        stop_words = trial.suggest_categorical("stop_words", [True, False])
        use_idf = trial.suggest_categorical("use_idf", [True, False])
        sublinear_tf = trial.suggest_categorical("sublinear_tf", [True, False])
        min_df = trial.suggest_int("min_df", 1, 10, step=1)
        max_df = trial.suggest_int("max_df", 0, 1, step=0.1)
        ngram_range = trial.suggest_categorical("ngram_range", [
            (1, 1), (1, 2), (1, 3), (1, 4), (1, 5),
            (2, 2), (2, 3), (2, 4), (2, 5),
            (3, 3)])

        tfidf_vectorizer = TfidfVectorizer(
            stop_words=stop_words, use_idf=use_idf, sublinear_tf=sublinear_tf, ngram_range=ngram_range, min_df=min_df, max_df=max_df)

        classifier_name = trial.suggest_categorical(
            "classifier", ["SVC", "RandomForest"])
        if classifier_name == "SVC":
            svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
            clf = SVC(C=svc_c, gamma="auto")
        else:
            rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
            clf = RandomForestClassifier(
                max_depth=rf_max_depth, n_estimators=10
            )
        pipe = make_pipeline(tfidf_vectorizer, clf)

        skf = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
        skf.get_n_splits(data, data[feat])

        cv_f1_scores = cross_val_score(
            estimator=pipe, X=x, y=y,
            n_jobs=-1, cv=skf, scoring='f1_macro')
        cv_f1_avg = cv_f1_scores.mean()
        return cv_f1_avg


for feat in upv_features:
    # print(feat)
    for camp, data in upv.groupby('¿Qué campaña?'):
        study = optuna.create_study(direction="maximize")
        study.optimize(objective_fn, n_trials=100)
        print(study.best_trial)

In [ ]:
# single campaign cross-validation: create folds
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

seed = 1234

upv_features = [
    # '1. Espacio',
    # '2. Texto', '3. Multimedia', '4. Elemento desinformador',
    # '5.  Tipo multimedia', '6. Alteración multimedia',
    '7. Cuenta emisora', '8. Fuente',
    '9. Protagonista',
    '9. Protagonista reduced',
    '10. Atributo',
    '11. Macro tema', '12. Populismo',
    '13. Ataque', '14. Tipo de ataque'
]

folds_output = 'cv_fold_indices.json'
cv_results_output = 'results_RF_CV.json'

results = []
cv_fold_indices = {}
for feat in upv_features:
    # print(feat)
    cv_fold_indices[feat] = {}
    for camp, data in upv.groupby('¿Qué campaña?'):
        cv_fold_indices[feat][camp] = {}
        # print(camp)
        skf = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
        skf.get_n_splits(data, data[feat])
        # Generate indices to split data into training and test set.
        for i, (train_index, test_index) in enumerate(skf.split(data, data[feat])):
            # convert ordinal indices to dataset indices
            train_index = data.iloc[train_index].index.tolist()
            test_index = data.iloc[test_index].index.tolist()
            cv_fold_indices[feat][camp][f"fold_{i}"] = {
                'train_indices': train_index,
                'test_indices': test_index
            }
        # do cv
        cv_f1_scores = cross_val_score(
            pipe, data['text'], data[feat], cv=skf, scoring='f1_macro')
        cv_f1_avg = cv_f1_scores.mean()
        cv_accuracy_scores = cross_val_score(
            pipe, data['text'], data[feat], cv=skf, scoring='accuracy')
        cv_accuracy_avg = cv_accuracy_scores.mean()
        print(feat, camp, "cv_f1_avg:", cv_f1_avg)
        results.append({
            'feat': feat,
            'train': camp,
            'test': camp,
            'accuracy': cv_accuracy_avg, 'f1': cv_f1_avg,
            'clf': 'RF_CV'
        })

if folds_output:
    with open(folds_output, 'w+') as f:
        json.dump(cv_fold_indices, f)

if cv_results_output:
    with open(cv_results_output, 'w+') as f:
        # json.dump(results, f, indent=4)
        print("[", file=f)
        for record in results[:-1]:
            print(f"    {json.dumps(record)},", file=f)
        for record in results[-1:]:
            print(f"    {json.dumps(record)}", file=f)
        print("]", file=f)

In [3]:
# train over all previous campaigns and their combinations
from make_train_test_combinations import train_test_combos, train_test_combos_all

train_test_combos = train_test_combos_all

upv_features = [
    # '1. Espacio',
    # '2. Texto', '3. Multimedia', '4. Elemento desinformador',
    # '5.  Tipo multimedia', '6. Alteración multimedia',
    # '7. Cuenta emisora', '8. Fuente',
    '9. Protagonista',
    '9. Protagonista reduced',
    '10. Atributo',
    '11. Macro tema', '12. Populismo',
    '13. Ataque', '14. Tipo de ataque'
]

results = []
for anotation in upv_features:
    selection = upv[upv[anotation].notnull()].copy()
    selection['label'] = selection.loc[:, anotation]
    data = {}
    for cs_train in train_test_combos:
        cs_train_list = [cs_train] if isinstance(
            cs_train, str) else cs_train
        data['train'] = selection[selection['¿Qué campaña?'].isin(
            cs_train_list)]

        pipe.fit(data['train']['text'], data['train']['label'])

        cs_test = train_test_combos[cs_train]
        for c in cs_test:
            train_test_repr = f'train: {'+'.join(cs_train)}, test:{c}'
            # print(train_test_repr)
            data['test'] = selection[selection['¿Qué campaña?'].isin(
                [c])]
            preds = pipe.predict(data['test']['text'])
            accuracy = np.mean(preds == data['test'][anotation])
            try:
                f1 = f1_score(y_true=data['test'][anotation], y_pred=preds)
            except:
                f1 = f1_score(y_true=data['test'][anotation],
                              y_pred=preds, average='macro')
            cf = confusion_matrix(y_true=data['test'][anotation], y_pred=preds)
            # print('dimensión', anotation)
            # print(f'accuracy: {accuracy}', f'f1: {f1}')
            test_indices = data['test'].index.to_list()
            index_name = data['test'].index.name
            output_preds = [{index_name: int(i), "prediction": str(p)}
                            for i, p in zip(test_indices, preds)]

            results.append({
                'feat': anotation,
                'train': cs_train,
                'test': c,
                'accuracy': accuracy, 'f1': f1, 'cf': cf.tolist(),
                'preds': output_preds})

print(*results, sep='\n')


with open('results_RF.json', 'w+') as f:
    # json.dump(results, f, indent=4)
    print("[", file=f)
    for record in results[:-1]:
        print(f"    {json.dumps(record)},", file=f)
    for record in results[-1:]:
        print(f"    {json.dumps(record)}", file=f)
    print("]", file=f)

Loaded UPV dataset from UPV_allURLs.csv
{'feat': '9. Protagonista', 'train': 'Gen19', 'test': 'Gal-Eus20', 'accuracy': 0.2682926829268293, 'f1': 0.04081632653061224, 'cf': [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 11, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 6, 0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 4, 1, 0, 0, 0, 0, 0, 0, 0]], 'preds': [{'ID_Medio_Crono': 350, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}, {'ID_Medio_Crono': 652, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}, {'ID_Medio_Crono': 346, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}, {'ID_Medio_Crono': 351, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}, {'ID_Medio_Crono': 53, 'prediction': 'Gobiernos, partidos y dirigentes de izqu

In [11]:
print(*[pd.Series(record['preds']).value_counts()
      for record in results], sep='\n\n')

{'ID_Medio_Crono': 350, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}     1
{'ID_Medio_Crono': 652, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}     1
{'ID_Medio_Crono': 346, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}     1
{'ID_Medio_Crono': 351, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}     1
{'ID_Medio_Crono': 53, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}      1
{'ID_Medio_Crono': 49, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}      1
{'ID_Medio_Crono': 647, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}     1
{'ID_Medio_Crono': 661, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}     1
{'ID_Medio_Crono': 348, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}     1
{'ID_Medio_Crono': 352, 'prediction': 'Gobiernos, partidos y dirigentes de izquierda'}     1
{'ID_Medio_Crono': 347, 'prediction': 'Gobiernos, partidos y dirigente

In [14]:
print(pd.DataFrame(results).applymap(
    lambda x: x['accuracy']).to_latex(float_format="%.2f"))

\begin{tabular}{lrrrrr}
\toprule
 & Gen19/Gal-Eus20 & Gen19+Gal-Eus20/Cat21 & Gen19+Gal-Eus20+Cat21/Mad21 & Gen19+Gal-Eus20+Cat21+Mad21/CyL22 & Gen19+Gal-Eus20+Cat21+Mad21+CyL22/And22 \\
\midrule
1. Espacio & 0.78 & 0.84 & 0.91 & 0.96 & 0.92 \\
2. Texto & 0.76 & 0.86 & 0.93 & 0.98 & 0.90 \\
3. Multimedia & 0.71 & 0.74 & 0.66 & 0.89 & 0.85 \\
4. Elemento desinformador & 0.59 & 0.73 & 0.72 & 0.80 & 0.74 \\
5.  Tipo multimedia & 0.51 & 0.68 & 0.65 & 0.67 & 0.65 \\
6. Alteración multimedia & 0.49 & 0.67 & 0.62 & 0.67 & 0.76 \\
7. Cuenta emisora & 0.66 & 0.64 & 0.79 & 0.72 & 0.69 \\
8. Fuente & 0.63 & 0.78 & 0.77 & 0.91 & 0.85 \\
9. Protagonista & 0.27 & 0.44 & 0.35 & 0.50 & 0.53 \\
10. Atributo & 0.76 & 0.82 & 0.76 & 0.70 & 0.66 \\
11. Macro tema & 0.51 & 0.64 & 0.62 & 0.63 & 0.68 \\
12. Populismo & 0.46 & 0.54 & 0.65 & 0.61 & 0.56 \\
13. Ataque & 0.83 & 0.64 & 0.66 & 0.59 & 0.60 \\
14. Tipo de ataque & 0.68 & 0.52 & 0.62 & 0.39 & 0.48 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_65592/532255570.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(pd.DataFrame(results).applymap(


In [15]:
print(pd.DataFrame(results).applymap(
    lambda x: x['f1']).to_latex(float_format="%.2f"))

\begin{tabular}{lrrrrr}
\toprule
 & Gen19/Gal-Eus20 & Gen19+Gal-Eus20/Cat21 & Gen19+Gal-Eus20+Cat21/Mad21 & Gen19+Gal-Eus20+Cat21+Mad21/CyL22 & Gen19+Gal-Eus20+Cat21+Mad21+CyL22/And22 \\
\midrule
1. Espacio & 0.51 & 0.56 & 0.60 & 0.64 & 0.62 \\
2. Texto & 0.48 & 0.85 & 0.91 & 0.98 & 0.90 \\
3. Multimedia & 0.50 & 0.55 & 0.56 & 0.61 & 0.59 \\
4. Elemento desinformador & 0.50 & 0.66 & 0.66 & 0.68 & 0.61 \\
5.  Tipo multimedia & 0.16 & 0.34 & 0.29 & 0.46 & 0.22 \\
6. Alteración multimedia & 0.20 & 0.39 & 0.29 & 0.53 & 0.48 \\
7. Cuenta emisora & 0.18 & 0.21 & 0.27 & 0.28 & 0.29 \\
8. Fuente & 0.52 & 0.71 & 0.50 & 0.81 & 0.80 \\
9. Protagonista & 0.04 & 0.25 & 0.17 & 0.16 & 0.22 \\
10. Atributo & 0.29 & 0.30 & 0.29 & 0.27 & 0.27 \\
11. Macro tema & 0.25 & 0.42 & 0.44 & 0.25 & 0.29 \\
12. Populismo & 0.21 & 0.26 & 0.29 & 0.48 & 0.42 \\
13. Ataque & 0.81 & 0.62 & 0.64 & 0.58 & 0.60 \\
14. Tipo de ataque & 0.21 & 0.15 & 0.17 & 0.11 & 0.15 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_65592/1871650307.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(pd.DataFrame(results).applymap(


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from collections import defaultdict
import numpy as np


seed = 1234


vec = CountVectorizer(
    lowercase=False,
    # stop_words=stopwords_spanish,
    ngram_range=(1, 2),
    min_df=0.00,
    max_df=0.8
)
tfidf = TfidfTransformer()

clfs = {
    'MultinomialNB': MultinomialNB(),

    'RandomForest': RandomForestClassifier(),

    'SVM': SGDClassifier(loss='hinge', penalty='l2',
                         alpha=1e-3, random_state=seed,
                         max_iter=5, tol=None),

    'XGB': XGBClassifier(random_state=42, seed=2,
                         colsample_bytree=0.6, subsample=0.7)
}


upv_features = ['1. Espacio',
                '2. Texto', '3. Multimedia', '4. Elemento desinformador',
                '5.  Tipo multimedia', '6. Alteración multimedia', '7. Cuenta emisora',
                '8. Fuente', '9. Protagonista', '10. Atributo', '11. Macro tema',
                '12. Populismo', '13. Ataque', '14. Tipo de ataque']

results = {feat: {} for feat in upv_features}

for feat in upv_features:

    ds = upv[(upv['text'].notnull() & upv[feat].notnull())]
    data = {}
    data['train'], data['dev'] = train_test_split(ds)

    for clf_name, clf in clfs.items():
        pipe = make_pipeline(vec, tfidf, clf)
        try:
            if clf_name == 'XGB':
                label_map = dict(enumerate(
                    data['train'][feat].unique()
                ))
                label_map = {v: k for k, v in label_map.items()}

                data_train_mapped_label = data['train'][feat].map(
                    label_map, na_action='ignore')
                data_dev_mapped_label = data['dev'][feat].map(
                    label_map, na_action='ignore')

                pipe.fit(data['train']['text'], data_train_mapped_label)
                predicted = pipe.predict(data['dev']['text'])
                # add F measures
                accuracy = np.mean(predicted == data_dev_mapped_label)
            else:
                pipe.fit(data['train']['text'], data['train'][feat])
                predicted = pipe.predict(data['dev']['text'])
                accuracy = np.mean(predicted == data['dev'][feat])

            results[feat][clf_name] = accuracy
            print(feat, clf_name, accuracy)
            print('\n')
        except Exception as e:
            print(e)
            print(feat, clf_name)
            print(data.keys())